In [1]:
import numpy as np
import pandas as pd
import _pickle as pickle
from collections import defaultdict
import re
import ast

from bs4 import BeautifulSoup

import sys
import os
import json
import operator

#from keras.preprocessing.text import Tokenizer, text_to_word_sequence
#from keras.preprocessing.sequence import pad_sequences
#from keras.utils.np_utils import to_categorical

#from keras import backend as K
#from keras.engine.topology import Layer, InputSpec
#from keras import initializers

#from keras.optimizers import SGD

#from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, Dropout, Activation, Input, Flatten, Concatenate, GlobalAveragePooling1D, LSTM, GRU, Bidirectional, dot, multiply, Lambda, TimeDistributed, Masking

#from keras.models import Model, Sequential
#from keras.regularizers import l2,l1,l1_l2
#from keras.callbacks import Callback,EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight

#from keras.callbacks import Callback


#from keras.layers.normalization import BatchNormalization
from sklearn.svm import SVC, LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, classification_report,accuracy_score, auc, roc_curve, roc_auc_score

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from collections import OrderedDict
from lime.lime_text import LimeTextExplainer 
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import text

In [3]:
import pickle

In [4]:
import optunity.metrics

In [5]:
from scipy import sparse

In [6]:
from sklearn.metrics import average_precision_score

In [7]:
def load_data(data_name):
    f = open(data_name,'rb')
    data = pickle.load(f)
    return data

In [8]:
train_labels = load_data('/home/jujun/fraudprediction_10k/data/y_train_20200214')

In [9]:
test_labels = load_data('/home/jujun/fraudprediction_10k/data/y_test_20200214')

In [10]:
train_indecis = load_data('/home/jujun/fraudprediction_10k/data/indices_train_20200214')

In [11]:
test_indecis = load_data('/home/jujun/fraudprediction_10k/data/indices_test_20200214')

In [12]:
# doc_text = load_data('doc_20200206_2012')

In [13]:
# my_stop_words = text.ENGLISH_STOP_WORDS.union(["dd","PRON","-PRON-"])

In [14]:
# tfidf_vect = TfidfVectorizer(stop_words = my_stop_words, min_df=5)
# tm= tfidf_vect.fit_transform(doc_text)

In [15]:
#tm is from the SVM_baseline script
tm = sparse.load_npz("/home/jujun/fraudprediction_10k/data/tm_20200212.npz")

In [16]:
tm

<56288x69185 sparse matrix of type '<class 'numpy.float64'>'
	with 48325957 stored elements in Compressed Sparse Row format>

In [17]:
X_train = tm[train_indecis]
X_test = tm[test_indecis]

In [19]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [20]:
f = 0

aucs = ()
aucs_list = []

precisions = ()

recalls = ()

result = pd.DataFrame()

exp_weight= []

# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for train_index, test_index in kfold.split(X_train,labels_train):
#     print(' ')

#     f = f+1
#     print('molde_'+str(f))
#     print('train: ',train_index[0], ' / ', train_index[-1])
#     print('test: ',test_index[0],' / ', test_index[-1])
    #ModelName = './folder(6)/basicM_bestM_l12_'+str(f)
#     x_traink, x_val = tm[train_index], tm[test_index]
#     y_traink, y_val = labels_train[train_index].values, labels_train[test_index].values
    
class_weights_dict = {}
class_weights = class_weight.compute_class_weight('balanced',
                                             np.unique(train_labels),
                                             train_labels)
class_weights_dict[0] = class_weights[0]
class_weights_dict[1] = class_weights[1]
print(class_weights_dict)

sample_weights = []
for y in train_labels:
    if y == 1:
        sample_weights.append(class_weights[1])
    else: sample_weights.append(class_weights[0])

print(class_weights[1] in sample_weights)
    
    
nb = MultinomialNB()
nb.fit(X_train, train_labels, sample_weight = sample_weights)

pred_proba = nb.predict_proba(X_test)    
auc = roc_auc_score(test_labels, pred_proba[:,1])
    
    #plot_coefficients(nb, tfidf_vect.get_feature_names())
    
pred = nb.predict(X_test)
prfs = precision_recall_fscore_support(test_labels, pred)
ap = average_precision_score(test_labels, pred)


dict_temp = {
    'average_precision':ap,
    'auc':auc,'precision_label_0':prfs[0][0],'precision_label_1':prfs[0][1],
    'recall_label_0':prfs[1][0],'recall_label_1':prfs[1][1],
    'fscore_label_0':prfs[2][0],'fscore_label_1':prfs[2][1],
    'support_label_0':prfs[3][0],'support_label_1':prfs[3][1],
}
temp_table = pd.DataFrame(dict_temp,index=[f])
result = pd.concat([temp_table,result])

{0: 0.5045378151260504, 1: 55.592592592592595}
True


In [21]:
result

,auc,average_precision,fscore_label_0,fscore_label_1,precision_label_0,precision_label_1,recall_label_0,recall_label_1,support_label_0,support_label_1
0,0.896612,0.032754,0.899793,0.073703,0.997817,0.038627,0.819306,0.80198,11157,101
